In [1]:
import pymongo as mg
import pandas as pd

In [4]:
client = mg.MongoClient(host='mongodb://localhost:27017')

In [5]:
database = client['db_medicals']
database

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_medicals')

In [6]:
collection = database['RecurrenceOfSurgery']
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), 'db_medicals'), 'RecurrenceOfSurgery')

In [7]:
cursor = collection.find({})

In [8]:
data_list = list(cursor)

In [9]:
df_ROS = pd.DataFrame(data_list)
df_ROS.head(2)

,_id,Unnamed: 0,환자ID,Large Lymphocyte,Location of herniation,ODI,가족력,간질성폐질환,고혈압여부,과거수술횟수,...,Modic change,PI,PT,Seg Angle(raw),Vaccum disc,골밀도,디스크단면적,디스크위치,척추이동척도,척추전방위증
0,64d33876d50e73290b2fe355,0,1PT,22.8,3,51.0,0.0,0,0,0,...,3,51.6,36.6,14.4,0,-1.01,2048.5,4,Down,0
1,64d33876d50e73290b2fe356,1,2PT,44.9,4,26.0,0.0,0,0,0,...,0,40.8,7.2,17.8,0,-1.14,1753.1,4,Up,0


### * 분석에 필요 없는 컬럼을 drop 시키기 위해 컬럼 갯수, 정보 확인

In [11]:
df_ROS.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 53 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   _id                     1894 non-null   object 
 1   Unnamed: 0              1894 non-null   int64  
 2   환자ID                    1894 non-null   object 
 3   Large Lymphocyte        1894 non-null   float64
 4   Location of herniation  1894 non-null   int64  
 5   ODI                     462 non-null    float64
 6   가족력                     1843 non-null   float64
 7   간질성폐질환                  1894 non-null   int64  
 8   고혈압여부                   1894 non-null   int64  
 9   과거수술횟수                  1894 non-null   int64  
 10  당뇨여부                    1894 non-null   int64  
 11  말초동맥질환여부                1894 non-null   int64  
 12  빈혈여부                    1894 non-null   int64  
 13  성별                      1894 non-null   int64  
 14  스테로이드치료                 1894 non-null   

In [17]:
df_ROS['수술기법'].value_counts()

TELD    1673
IELD     140
Name: 수술기법, dtype: int64

### * 컬럼을 drop시키기 위해 컬럼명 확인 (디스크 관련 데이터 인것 같다)

In [20]:
df_ROS.columns

Index(['_id', 'Unnamed: 0', '환자ID', 'Large Lymphocyte',
       'Location of herniation', 'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수',
       '당뇨여부', '말초동맥질환여부', '빈혈여부', '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환',
       '암발병여부', '연령', '우울증여부', '입원기간', '입원일자', '종양진행여부', '직업', '체중', '퇴원일자',
       '헤모글로빈수치', '혈전합병증여부', '환자통증정도', '흡연여부', '통증기간(월)', '수술기법', '수술시간',
       '수술실패여부', '수술일자', '재발여부', '혈액형', '전방디스크높이(mm)', '후방디스크높이(mm)', '지방축적도',
       'Instability', 'MF + ES', 'Modic change', 'PI', 'PT', 'Seg Angle(raw)',
       'Vaccum disc', '골밀도', '디스크단면적', '디스크위치', '척추이동척도', '척추전방위증'],
      dtype='object')

### 나는 체중에 따른 디스크를 분석할것이다.

In [21]:
df_ROS_dropcol = df_ROS.drop(columns=['_id', 'Unnamed: 0', '환자ID', 'Large Lymphocyte',
       'Location of herniation', 'ODI', '가족력', '간질성폐질환', '고혈압여부', '과거수술횟수',
       '당뇨여부', '말초동맥질환여부', '빈혈여부', '성별', '스테로이드치료', '신부전여부', '신장', '심혈관질환',
       '암발병여부', '연령', '우울증여부', '입원기간', '입원일자', '종양진행여부', '직업','퇴원일자',
       '헤모글로빈수치', '혈전합병증여부', '환자통증정도', '흡연여부', '통증기간(월)','수술기법', '수술시간',
       '수술실패여부', '수술일자', '재발여부', '혈액형', 'Instability', 'MF + ES', 'Modic change', 'PI', 'PT', 'Seg Angle(raw)','Vaccum disc', ])

In [23]:
df_ROS_dropcol.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1894 entries, 0 to 1893
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   체중           1894 non-null   float64
 1   전방디스크높이(mm)  1894 non-null   float64
 2   후방디스크높이(mm)  1894 non-null   float64
 3   지방축적도        1891 non-null   float64
 4   골밀도          998 non-null    float64
 5   디스크단면적       1893 non-null   float64
 6   디스크위치        1894 non-null   int64  
 7   척추이동척도       1894 non-null   object 
 8   척추전방위증       1894 non-null   int64  
dtypes: float64(6), int64(2), object(1)
memory usage: 133.3+ KB


In [24]:
df_ROS_dropcol.describe(include=object)

,척추이동척도
count,1894
unique,5
top,Middle
freq,1260
